In [ ]:
from risq import *

import matplotlib.pyplot as plt

In [ ]:
# In the toy model, a cell can be in one of two states:
# - H: healthy
# - C: cancerous
# Healthy cells can become cancerous with a probability `prob_mutate`
# and cancerous cells can overgrow healthy cells with a probability `prob_spread`
def create_toy_model(prob_mutate: float, prob_spread: float):
    num_states = 2  # healthy and cancerous
    num_neighbors = 4  # in a 2 dimensional grid, each cell has 4 neighbors

    p = prob_mutate
    q = prob_spread

    probs_internal = [
        [1.0 - p, 0.0],  # H
        [   p   , 1.0],  # C
    ]

    probs_spread = [
        [0.0, 0.0],  # H
        [ q , 0.0],  # C
    ]

    return Model(
        num_states=num_states,
        num_neighbors=num_neighbors,
        probs_internal=probs_internal,
        probs_spread=probs_spread
    )

In [ ]:
def plot_figure(path: str, method: type[Method], prob_mutate: float, prob_spread: float):
    # Create a model with some parameters
    model = create_toy_model(
        prob_mutate=prob_mutate,
        prob_spread=prob_spread,
    )

    # Set time range
    time_steps = 20

    # State we are interested in plotting
    state = 1  # = C

    simulations: list[Method] = [
        MonteCarlo2D(model, **{
            "num_trials": 1000,
            "width": 20,
            "height": 20,
            "time_steps": time_steps,
            "seed": 42
        }),
        method(model),
    ]

    for simulation in simulations:
        print(f"🧬 Simulating using {simulation.__class__.__name__} ..")
        print(f"- mean =     {simulation.probability(time_steps, state)}")
        print(f"- variance = {simulation.variance(time_steps, state)}")
        print()

    plot_distributions(simulations, time_steps, state)
    plt.savefig(path)

    return simulations

In [ ]:
# Comparison Monte Carlo simulation to single cell approximation
plot_figure("img/monte_carlo_vs_single_cell_strong_interaction.svg", SingleCellMethod, 0.01, 0.05)
plot_figure("img/monte_carlo_vs_single_cell_weak_interaction.svg", SingleCellMethod, 0.05, 0.01)

# Comparison Monte Carlo simulation to neighboring cells approximation
plot_figure("img/monte_carlo_vs_neighboring_cells_strong_interaction.svg", NeighboringCellMethod, 0.01, 0.05)
plot_figure("img/monte_carlo_vs_neighboring_cells_weak_interaction.svg", NeighboringCellMethod, 0.05, 0.01)